1. Importing Libraries and Data Set

In [5]:
# Importing Libraries
import pandas as pd
import requests

ModuleNotFoundError: No module named 'urllib3.packages.six.moves'

In [32]:
# Load the dataset
file_path = "critic_reviews.csv"  # Update the path if necessary
df = pd.read_csv(file_path)

2. Inspecting the Dataset

In [ ]:
print(df.head())          # View the first few rows
print(df.info())          # Check column data types and non-null values
print(df.describe())      # Summary statistics for numerical columns
print(df.isnull().sum())  # Count of missing values per column

3. Data Cleaning

In [ ]:
# Handling Missing Values
df['criticName'] = df['criticName'].fillna("n/a")
df['criticPageUrl'] = df['criticPageUrl'].fillna("n/a")

# Using a placehold for missing numerical values
df['originalScore'] = df['originalScore'].fillna(0)  # Replace with 0

# Dropping irrelevant columns with missing values
df = df.drop(columns = ['isRtUrl', 'reviewUrl', 'quote', 'publicationUrl'], errors = 'ignore')

print(df.isnull().sum()) # Testing to make sure missing values are handled

In [35]:
# Handling different inputs for originalScore

def clean_original_score(value):
    # Handle missing or placeholder values
    if value == "-1" or pd.isnull(value):
        return None  # Replace with NULL for SQL
    
    # Handle fractions like "2.5/4" or "76/100"
    if isinstance(value, str) and "/" in value:
        try:
            numerator, denominator = map(float, value.split("/"))
            return numerator / denominator  # Convert to decimal
        except ValueError:
            return None  # If it fails to split correctly
    
    # Handle letter grades
    letter_to_score = {
        "A": 4.0, "A-": 3.7, "B+": 3.3, "B": 3.0, "B-": 2.7,
        "C+": 2.3, "C": 2.0, "C-": 1.7, "D": 1.0, "F": 0.0
    }
    if value in letter_to_score:
        return letter_to_score[value]
    
    # Handle date-like values (e.g., "5-Apr")
    months = ["jan", "feb", "mar", "apr", "may", "jun", "jul", "aug", "sep", "oct", "nov", "dec"]
    if isinstance(value, str) and any(month in value.lower() for month in months):
        return None  # Replace with NULL for SQL
    
    # Handle direct numerical scores
    try:
        return float(value)
    except ValueError:
        return None  # If it can't be converted, replace with NULL

df["originalScore"] = df["originalScore"].apply(clean_original_score)

In [36]:
# Checking for Duplicates
print(df.duplicated(subset=['reviewId']).sum())

0


In [37]:
# Standardizing Columns
df['criticName'] = df['criticName'].str.lower()                 #Converting to lowercase for uniformity
df['publicationName'] = df['publicationName'].str.lower()       #Converting to lowercase for uniformity


In [38]:
# Exporting cleaned dataset
df.to_csv("cleaned_originalScore.csv", index=False)